In [32]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from typing import Dict, Tuple
import os
import glob
import xlsxwriter
import smtplib
from datetime import datetime, timedelta
from email.message import EmailMessage

In [13]:
def load_locations_from_csv(file_path: str) -> Dict[str, Tuple[float, float]]:
    """
    Load store locations from a CSV file and return a dictionary of locations.

    Args:
        file_path: Path to the CSV file

    Returns:
        Dictionary with store names as keys and (latitude, longitude) tuples as values
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Convert the DataFrame into a dictionary
    locations = dict(zip(df['Location: Name (Grouped)'], zip(df['Latitude'], df['Longitude'])))
    
    return locations

# Import Coordinates
file_path = 'Store_Coordinates.csv'
locations = load_locations_from_csv(file_path)

In [60]:
def get_weather_data(locations: Dict[str, Tuple[float, float]], date_range: list) -> pd.DataFrame:
    """
    Fetch temperature data for multiple locations and return a single DataFrame.
    
    Args:
        locations: Dictionary with location names as keys and (latitude, longitude) tuples as values
    
    Returns:
        DataFrame with date index and location temperatures as columns
    """
    # Setup the Open-Meteo API client
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)
    
    all_data = {}
    
    # Get the first location's data to establish the date range
    first_location = next(iter(locations))
    first_lat, first_lon = locations[first_location]
    params = {
        "latitude": first_lat,
        "longitude": first_lon,
        "start_date": date_range[0],
        "end_date": date_range[1], 
        "hourly": "temperature_2m",
        "daily": ("weather_code","rain_sum","snowfall_sum","apparent_temperature_max"),
        "temperature_unit": "fahrenheit"
    }
    
    response = openmeteo.weather_api("https://api.open-meteo.com/v1/forecast", params=params)[0]
    hourly = response.Hourly()
    
    # Create the date index
    daily_dates = pd.date_range(start=date_range[0], end=date_range[1])
    # Create empty DataFrames
    daily_averages_list = []
    
    # Create DataFrame with date index
    daily_averages_df = pd.DataFrame(index=daily_dates)
    weather_code_df = pd.DataFrame(index=daily_dates)
    rain_df = pd.DataFrame(index=daily_dates)
    snowfall_df = pd.DataFrame(index=daily_dates)
    temp_max_df = pd.DataFrame(index=daily_dates)
    
    # Fetch data for each location
    for location_name, (lat, lon) in locations.items():
        params = {
            "latitude": lat,
            "longitude": lon,
            "timezone": "auto",
            "start_date": date_range[0],
            "end_date": date_range[1],
            "hourly": "temperature_2m",
            "daily": ("weather_code","rain_sum","snowfall_sum","apparent_temperature_max"),
            "temperature_unit": "fahrenheit"
        }
        
        response = openmeteo.weather_api("https://api.open-meteo.com/v1/forecast", params=params)[0]
        hourly = response.Hourly()
        daily = response.Daily()
        temperature_2m = hourly.Variables(0).ValuesAsNumpy()
        weather_code = daily.Variables(0).ValuesAsNumpy()
        rain_sum = daily.Variables(1).ValuesAsNumpy()
        snowfall_sum = daily.Variables(2).ValuesAsNumpy()
        apparent_temperature_max = daily.Variables(3).ValuesAsNumpy()

        
        # Compute daily averages
        daily_avg_temp = daily_averages(temperature_2m)
        
        # Store in list for long format conversion
        for date, temp in zip(daily_dates, daily_avg_temp):
            daily_averages_list.append([location_name, date, temp])

        # Convert to DataFrame in long format
        daily_averages_df = pd.DataFrame(daily_averages_list, columns=["Location", "Date", "Temperature"])

        # Add location data as a new column
        weather_code_df[location_name] = weather_code
        rain_df[location_name] = rain_sum
        snowfall_df[location_name] = snowfall_sum
        temp_max_df[location_name] = apparent_temperature_max

        weather_data = {
            "Daily_Average": daily_averages_df,
            "Weather_Code": weather_code_df,
            "Total_Rain": rain_df,
            "Total_Snowfall": snowfall_df,
            "Max_Temp": temp_max_df
        }
    return weather_data

In [20]:
def daily_averages(temperature_2m):
    """
    Calculate average temperature during store hours (11am-7pm)

    Args:
        temperature_2m: list of temperatures by hour 

    Returns:
        daily_averages: list of average temperatures each day during store hours
    """
    
    num_days = len(temperature_2m) // 24  # Ensure there are 7 full days
    daily_averages = []
    
    for day in range(num_days):
        start_idx = day * 24 + 10  # Index for 10:00 AM
        end_idx = day * 24 + 20  # Index for 19:00 (exclusive)
        avg_temp = sum(temperature_2m[start_idx:end_idx]) / (end_idx - start_idx)
        daily_averages.append(avg_temp)
    
    return daily_averages


In [52]:
def get_week_range():
    today = datetime.today()
    last_monday = today - timedelta(days=today.weekday() + 7)  # Previous Monday
    last_sunday = last_monday + timedelta(days=6)  # Following Sunday
    return [last_monday.strftime('%Y-%m-%d'), last_sunday.strftime('%Y-%m-%d')]

In [79]:
def send_email(file_path):
    sender_email = "ryanyick@marinelayer.com"
    receiver_emails = ["ryanyick@marinelayer.com"]
    subject = "Weekly Weather Report"
    body = "Attached is the weekly weather report."
    
    msg = EmailMessage()
    msg['From'] = sender_email
    msg['To'] = ", ".join(receiver_emails)
    msg['Subject'] = subject
    msg.set_content(body)
    
    with open(file_path, "rb") as f:
        msg.add_attachment(f.read(), maintype='application', subtype='xlsx', filename=file_path)
    
    with smtplib.SMTP('smtp.gmail.com', 587) as server: 
        server.starttls()
        server.login("ryanyick@marinelayer.com", "lwbc uvqv jobv exfi")  # Use app password if needed
        server.send_message(msg)

In [83]:
def fetch_weather_data():
    date_range = get_week_range()

    # Import Coordinates
    file_path = 'Store_Coordinates.csv'
    locations = load_locations_from_csv(file_path)

    #Run Function
    weather_data = get_weather_data(locations, date_range)
    
    # Save to file
    output_file = "weather_report.xlsx"
    with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
        for sheet_name, df in weather_data.items():  # Loop through the dictionary
            # Convert index to timezone-unaware datetime (if it contains timezone info)
            if isinstance(df.index, pd.DatetimeIndex):
                df.index = df.index.tz_localize(None)  # Remove timezone
                
            df.to_excel(writer, sheet_name=sheet_name, index=True)  # Save each DataFrame
    
    send_email(output_file)